In [1]:
import pandas as pd
import numpy as np

In [12]:
import datetime as dt

In [105]:
df = pd.read_csv('https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-01.csv')

In [106]:
df.shape

(7667792, 18)

In [107]:
# drop all cash payments as they do not record tips

df = df[df.payment_type != 2]

In [108]:
df.shape

(5530377, 18)

In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5530377 entries, 0 to 7667791
Data columns (total 18 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VendorID               int64  
 1   tpep_pickup_datetime   object 
 2   tpep_dropoff_datetime  object 
 3   passenger_count        int64  
 4   trip_distance          float64
 5   RatecodeID             int64  
 6   store_and_fwd_flag     object 
 7   PULocationID           int64  
 8   DOLocationID           int64  
 9   payment_type           int64  
 10  fare_amount            float64
 11  extra                  float64
 12  mta_tax                float64
 13  tip_amount             float64
 14  tolls_amount           float64
 15  improvement_surcharge  float64
 16  total_amount           float64
 17  congestion_surcharge   float64
dtypes: float64(9), int64(6), object(3)
memory usage: 801.7+ MB


In [110]:
df.drop(columns = ['congestion_surcharge','store_and_fwd_flag'], inplace = True)

In [111]:
df.columns = df.columns.str.lower()

In [112]:
df.rename(columns={'vendorid':'vendor_id', 'tpep_pickup_datetime':'pickup_datetime',
                  'tpep_dropoff_datetime':'dropoff_datetime'},inplace=True)

In [113]:
df.pickup_datetime, df.dropoff_datetime = pd.to_datetime(df.pickup_datetime), pd.to_datetime(df.dropoff_datetime)

In [114]:
df['week_of_month'] = df.pickup_datetime.dt.weekofyear

<ipython-input-114-369b3419fa04>:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['week_of_month'] = df.pickup_datetime.dt.weekofyear


In [115]:
df.week_of_month.value_counts()

2     1295391
4     1291721
3     1288264
1      861224
5      793749
52          6
12          3
27          3
17          2
9           2
6           2
24          2
21          1
15          1
51          1
8           1
7           1
30          1
36          1
11          1
Name: week_of_month, dtype: int64

In [116]:
#removing erroneous dates for accuracy.
df = df[df.week_of_month<6]

In [117]:
df.week_of_month.value_counts()

2    1295391
4    1291721
3    1288264
1     861224
5     793749
Name: week_of_month, dtype: int64

In [118]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5530349 entries, 0 to 7667791
Data columns (total 17 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   vendor_id              int64         
 1   pickup_datetime        datetime64[ns]
 2   dropoff_datetime       datetime64[ns]
 3   passenger_count        int64         
 4   trip_distance          float64       
 5   ratecodeid             int64         
 6   pulocationid           int64         
 7   dolocationid           int64         
 8   payment_type           int64         
 9   fare_amount            float64       
 10  extra                  float64       
 11  mta_tax                float64       
 12  tip_amount             float64       
 13  tolls_amount           float64       
 14  improvement_surcharge  float64       
 15  total_amount           float64       
 16  week_of_month          int64         
dtypes: datetime64[ns](2), float64(8), int64(7)
memory usage: 759.5 MB

In [119]:
df.shape

(5530349, 17)

In [123]:
df.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,ratecodeid,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,week_of_month
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,1
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,1,239,246,1,14.0,0.5,0.5,1.00,0.0,0.3,16.30,1
7,1,2019-01-01 00:21:28,2019-01-01 00:28:37,1,1.3,1,163,229,1,6.5,0.5,0.5,1.25,0.0,0.3,9.05,1
8,1,2019-01-01 00:32:01,2019-01-01 00:45:39,1,3.7,1,229,7,1,13.5,0.5,0.5,3.70,0.0,0.3,18.50,1
9,1,2019-01-01 00:57:32,2019-01-01 01:09:32,2,2.1,1,141,234,1,10.0,0.5,0.5,1.70,0.0,0.3,13.00,1


In [124]:
#renaming tip column and placing it first

df.rename(columns={'tip_amount':'tip'},inplace=True)

In [125]:
cols =list(df.columns)

In [126]:
cols.insert(0, cols.pop(cols.index('tip')))

In [127]:
df = df[cols]

In [128]:
df.head(1)

,tip,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,ratecodeid,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tolls_amount,improvement_surcharge,total_amount,week_of_month
0,1.65,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,151,239,1,7.0,0.5,0.5,0.0,0.3,9.95,1


In [129]:
#function to make tip column binary


def binary_creator(value):
    if value > 0:
        return 1
    else:
        return 0

In [130]:
df.tip = df.tip.apply(binary_creator)

In [131]:
df.tip.value_counts()

1    5257054
0     273295
Name: tip, dtype: int64

In [132]:
df['pickup_hour'] = df.pickup_datetime.dt.hour

In [133]:
df.head(1)

,tip,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,ratecodeid,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tolls_amount,improvement_surcharge,total_amount,week_of_month,pickup_hour
0,1,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,151,239,1,7.0,0.5,0.5,0.0,0.3,9.95,1,0


In [134]:
df.pickup_hour.value_counts()

18    383052
19    357063
17    338498
20    321151
15    312314
21    309194
14    297060
16    294121
8     286259
13    277294
22    276478
12    276043
9     270403
11    258960
10    253830
7     227612
23    204955
0     147405
6     122361
1     105743
2      76363
3      51607
5      47246
4      35337
Name: pickup_hour, dtype: int64

In [142]:
df['day'] = df.pickup_datetime.dt.day_name()

In [146]:
df.head(1)

,tip,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,ratecodeid,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tolls_amount,improvement_surcharge,total_amount,week_of_month,pickup_hour,day
0,1,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,151,239,1,7.0,0.5,0.5,0.0,0.3,9.95,1,0,Tuesday


In [144]:
# set day as a string to make it categorical

df.day = df.day.astype(str)

In [147]:
df.day.value_counts()

Thursday     993385
Wednesday    918705
Tuesday      869450
Friday       781548
Saturday     704813
Monday       655878
Sunday       606570
Name: day, dtype: int64

In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5530349 entries, 0 to 7667791
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   tip                    int64         
 1   vendor_id              int64         
 2   pickup_datetime        datetime64[ns]
 3   dropoff_datetime       datetime64[ns]
 4   passenger_count        int64         
 5   trip_distance          float64       
 6   ratecodeid             int64         
 7   pulocationid           int64         
 8   dolocationid           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tolls_amount           float64       
 14  improvement_surcharge  float64       
 15  total_amount           float64       
 16  week_of_month          int64         
 17  pickup_hour            int64         
 18  day                   

In [136]:
df.head(1)

,tip,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,ratecodeid,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tolls_amount,improvement_surcharge,total_amount,week_of_month,pickup_hour,day
0,1,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,151,239,1,7.0,0.5,0.5,0.0,0.3,9.95,1,0,Tuesday


In [137]:
locationdf = pd.read_csv('data/locations.csv')

In [66]:
locationdf

,locationid,borough,zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,NV,NaN


In [148]:
locationdf.columns = locationdf.columns.str.lower()

In [149]:
df.rename(columns={'pulocationid':'locationid'}, inplace=True)

In [150]:
df = pd.merge(df, locationdf[['locationid','borough','zone']], how='left', on='locationid')

In [151]:
df.head()

,tip,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,ratecodeid,locationid,dolocationid,payment_type,...,extra,mta_tax,tolls_amount,improvement_surcharge,total_amount,week_of_month,pickup_hour,day,borough,zone
0,1,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,151,239,1,...,0.5,0.5,0.0,0.3,9.95,1,0,Tuesday,Manhattan,Manhattan Valley
1,1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,1,239,246,1,...,0.5,0.5,0.0,0.3,16.30,1,0,Tuesday,Manhattan,Upper West Side South
2,1,1,2019-01-01 00:21:28,2019-01-01 00:28:37,1,1.3,1,163,229,1,...,0.5,0.5,0.0,0.3,9.05,1,0,Tuesday,Manhattan,Midtown North
3,1,1,2019-01-01 00:32:01,2019-01-01 00:45:39,1,3.7,1,229,7,1,...,0.5,0.5,0.0,0.3,18.50,1,0,Tuesday,Manhattan,Sutton Place/Turtle Bay North
4,1,1,2019-01-01 00:57:32,2019-01-01 01:09:32,2,2.1,1,141,234,1,...,0.5,0.5,0.0,0.3,13.00,1,0,Tuesday,Manhattan,Lenox Hill West


In [152]:
df.rename(columns={'borough':'pickup_borough', 'locationid':'pickup_location_id',
                  'dolocationid':'locationid','zone':'pickup_zone'}, inplace=True)

In [153]:
df.head()

,tip,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,ratecodeid,pickup_location_id,locationid,payment_type,...,extra,mta_tax,tolls_amount,improvement_surcharge,total_amount,week_of_month,pickup_hour,day,pickup_borough,pickup_zone
0,1,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,151,239,1,...,0.5,0.5,0.0,0.3,9.95,1,0,Tuesday,Manhattan,Manhattan Valley
1,1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,1,239,246,1,...,0.5,0.5,0.0,0.3,16.30,1,0,Tuesday,Manhattan,Upper West Side South
2,1,1,2019-01-01 00:21:28,2019-01-01 00:28:37,1,1.3,1,163,229,1,...,0.5,0.5,0.0,0.3,9.05,1,0,Tuesday,Manhattan,Midtown North
3,1,1,2019-01-01 00:32:01,2019-01-01 00:45:39,1,3.7,1,229,7,1,...,0.5,0.5,0.0,0.3,18.50,1,0,Tuesday,Manhattan,Sutton Place/Turtle Bay North
4,1,1,2019-01-01 00:57:32,2019-01-01 01:09:32,2,2.1,1,141,234,1,...,0.5,0.5,0.0,0.3,13.00,1,0,Tuesday,Manhattan,Lenox Hill West


In [169]:
df = pd.merge(df, locationdf[['locationid','borough','zone']], how='left', on='locationid')
df.rename(columns={'borough':'dropoff_borough','locationid':'dropoff_location_id',
                  'zone':'dropoff_zone'},inplace=True)

In [172]:
df.head(1)

,tip,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,ratecodeid,pickup_location_id,dropoff_location_id,payment_type,...,tolls_amount,improvement_surcharge,total_amount,week_of_month,pickup_hour,day,pickup_borough,pickup_zone,dropoff_borough,dropoff_zone
0,1,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,151,239,1,...,0.0,0.3,9.95,1,0,Tuesday,Manhattan,Manhattan Valley,Manhattan,Upper West Side South


In [174]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5530349 entries, 0 to 5530348
Data columns (total 23 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   tip                    int64         
 1   vendor_id              int64         
 2   pickup_datetime        datetime64[ns]
 3   dropoff_datetime       datetime64[ns]
 4   passenger_count        int64         
 5   trip_distance          float64       
 6   ratecodeid             int64         
 7   pickup_location_id     int64         
 8   dropoff_location_id    int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tolls_amount           float64       
 14  improvement_surcharge  float64       
 15  total_amount           float64       
 16  week_of_month          int64         
 17  pickup_hour            int64         
 18  day                   

In [175]:
df.day = df.day.astype(str)
df.pickup_borough = df.pickup_borough.astype(str)
df.pickup_zone = df.pickup_zone.astype(str)
df.dropoff_borough = df.dropoff_borough.astype(str)
df.dropoff_zone = df.dropoff_zone.astype(str)
df.week_of_month = df.week_of_month.astype(str)
df.pickup_hour = df.pickup_hour.astype(str)

In [184]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5530349 entries, 0 to 5530348
Data columns (total 23 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   tip                    int64         
 1   vendor_id              int64         
 2   pickup_datetime        datetime64[ns]
 3   dropoff_datetime       datetime64[ns]
 4   passenger_count        int64         
 5   trip_distance          float64       
 6   ratecodeid             int64         
 7   pickup_location_id     int64         
 8   dropoff_location_id    int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tolls_amount           float64       
 14  improvement_surcharge  float64       
 15  total_amount           float64       
 16  week_of_month          object        
 17  pickup_hour            object        
 18  day                   

In [183]:
df.week_of_month = df.week_of_month.astype(str)


In [185]:
df.head(1)

,tip,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,ratecodeid,pickup_location_id,dropoff_location_id,payment_type,...,tolls_amount,improvement_surcharge,total_amount,week_of_month,pickup_hour,day,pickup_borough,pickup_zone,dropoff_borough,dropoff_zone
0,1,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,151,239,1,...,0.0,0.3,9.95,1,0,Tuesday,Manhattan,Manhattan Valley,Manhattan,Upper West Side South


In [191]:
dummies = pd.get_dummies(data = df, columns=['week_of_month','pickup_hour','day','pickup_borough',
                                  'pickup_zone','dropoff_borough','dropoff_zone'], drop_first=True)

In [188]:
df.columns

Index(['tip', 'vendor_id', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'trip_distance', 'ratecodeid', 'pickup_location_id',
       'dropoff_location_id', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tolls_amount', 'improvement_surcharge', 'total_amount',
       'week_of_month', 'pickup_hour', 'day', 'pickup_borough', 'pickup_zone',
       'dropoff_borough', 'dropoff_zone'],
      dtype='object')

In [189]:
df.head()

,tip,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,ratecodeid,pickup_location_id,dropoff_location_id,payment_type,...,tolls_amount,improvement_surcharge,total_amount,week_of_month,pickup_hour,day,pickup_borough,pickup_zone,dropoff_borough,dropoff_zone
0,1,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,151,239,1,...,0.0,0.3,9.95,1,0,Tuesday,Manhattan,Manhattan Valley,Manhattan,Upper West Side South
1,1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,1,239,246,1,...,0.0,0.3,16.30,1,0,Tuesday,Manhattan,Upper West Side South,Manhattan,West Chelsea/Hudson Yards
2,1,1,2019-01-01 00:21:28,2019-01-01 00:28:37,1,1.3,1,163,229,1,...,0.0,0.3,9.05,1,0,Tuesday,Manhattan,Midtown North,Manhattan,Sutton Place/Turtle Bay North
3,1,1,2019-01-01 00:32:01,2019-01-01 00:45:39,1,3.7,1,229,7,1,...,0.0,0.3,18.50,1,0,Tuesday,Manhattan,Sutton Place/Turtle Bay North,Queens,Astoria
4,1,1,2019-01-01 00:57:32,2019-01-01 01:09:32,2,2.1,1,141,234,1,...,0.0,0.3,13.00,1,0,Tuesday,Manhattan,Lenox Hill West,Manhattan,Union Sq


In [196]:
df.to_csv(r'/Users/MichaelHarnett/Desktop/Metis/Projects/taxi_tip_classification_metis4/Data/\yellow1.csv')

In [192]:
dummies

,tip,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,ratecodeid,pickup_location_id,dropoff_location_id,payment_type,...,dropoff_zone_Williamsburg (North Side),dropoff_zone_Williamsburg (South Side),dropoff_zone_Windsor Terrace,dropoff_zone_Woodhaven,dropoff_zone_Woodlawn/Wakefield,dropoff_zone_Woodside,dropoff_zone_World Trade Center,dropoff_zone_Yorkville East,dropoff_zone_Yorkville West,dropoff_zone_nan
0,1,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.50,1,151,239,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.60,1,239,246,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,2019-01-01 00:21:28,2019-01-01 00:28:37,1,1.30,1,163,229,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,2019-01-01 00:32:01,2019-01-01 00:45:39,1,3.70,1,229,7,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,2019-01-01 00:57:32,2019-01-01 01:09:32,2,2.10,1,141,234,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5530344,1,2,2019-01-31 23:57:36,2019-02-01 00:18:39,1,4.79,1,263,4,1,...,0,0,0,0,0,0,0,0,0,0
5530345,0,2,2019-01-31 23:32:03,2019-01-31 23:33:11,1,0.00,1,193,193,1,...,0,0,0,0,0,0,0,0,0,0
5530346,0,2,2019-01-31 23:36:36,2019-01-31 23:36:40,1,0.00,1,264,264,1,...,0,0,0,0,0,0,0,0,0,0
5530347,0,2,2019-01-31 23:14:53,2019-01-31 23:15:20,1,0.00,1,264,7,1,...,0,0,0,0,0,0,0,0,0,0


In [194]:
dummies.columns

Index(['tip', 'vendor_id', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'trip_distance', 'ratecodeid', 'pickup_location_id',
       'dropoff_location_id', 'payment_type',
       ...
       'dropoff_zone_Williamsburg (North Side)',
       'dropoff_zone_Williamsburg (South Side)',
       'dropoff_zone_Windsor Terrace', 'dropoff_zone_Woodhaven',
       'dropoff_zone_Woodlawn/Wakefield', 'dropoff_zone_Woodside',
       'dropoff_zone_World Trade Center', 'dropoff_zone_Yorkville East',
       'dropoff_zone_Yorkville West', 'dropoff_zone_nan'],
      dtype='object', length=580)

In [197]:
dummies.to_csv(r'/Users/MichaelHarnett/Desktop/Metis/Projects/taxi_tip_classification_metis4/Data/\yellow_dummies.csv', index=False)

In [198]:
dumies

NameError: name 'dumies' is not defined